In [8]:
import cv2
import numpy as np

# Global variables
drawing = False
last_x, last_y = -1, -1
color = (255, 255, 255)

def draw_line(event, x, y, flags, param):
    global drawing, last_x, last_y, color

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        last_x, last_y = x, y
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            draw_line_on_canvas(param, x, y)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        draw_line_on_canvas(param, x, y)

def draw_line_on_canvas(canvas, x, y):
    global last_x, last_y
    cv2.line(canvas, (last_x, last_y), (x, y), color, 5)
    last_x, last_y = x, y

def choose_color(x, y):
    global color

    if 50 <= x <= 150 and 0 <= y <= 50:
        color = (255, 0, 0)  # Red
    elif 200 <= x <= 300 and 0 <= y <= 50:
        color = (0, 255, 0)  # Green
    elif 350 <= x <= 450 and 0 <= y <= 50:
        color = (0, 0, 255)  # Blue
    elif 500 <= x <= 600 and 0 <= y <= 50:
        color = (0, 255, 255)  # Yellow

# Create a black canvas
canvas = np.zeros((300, 400, 3), dtype=np.uint8)

# Set up the window and callback
cv2.namedWindow('Digital Whiteboard')
cv2.setMouseCallback('Digital Whiteboard', draw_line, canvas)

# Capture the camera frame once outside the loop
cap = cv2.VideoCapture(0)

# Main loop
while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (800, 600))

    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):
        canvas = np.zeros((300, 400, 3), dtype=np.uint8)
    elif key == ord('q'):
        break

    # Draw color selection buttons
    cv2.rectangle(frame, (50, 0), (150, 50), (255, 0, 0), -1)
    cv2.rectangle(frame, (200, 0), (300, 50), (0, 255, 0), -1)
    cv2.rectangle(frame, (350, 0), (450, 50), (0, 0, 255), -1)
    cv2.rectangle(frame, (500, 0), (600, 50), (0, 255, 255), -1)

    # Check if fingers are on color selection buttons
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    for i in range(50, 601, 150):
        choose_color(i, 25)
        roi = hsv_frame[0:50, i-50:i]
        if np.sum(roi) > 0:
            cv2.rectangle(frame, (i-50, 0), (i, 50), color, 2)

    # Display canvas and camera feed
    cv2.imshow('Digital Whiteboard', canvas)
    cv2.imshow('Camera', frame)

cap.release()
cv2.destroyAllWindows()


In [3]:
import numpy as np
import cv2
from collections import deque